In [269]:
import icf
import matplotlib.pyplot as plt
import numpy as np
name_list = ["horizontal.csv","45_degrees.csv","Vertical.csv","135_Degrees.csv"]
xdata = []
ydata = []
for n in  range(0,len(name_list)):
    xd, yd = icf.load_2col(name_list[n])
    xdata.append(xd)
    ydata.append(yd)


In [270]:
for x in range (0,len(xdata)):
    xdata[x] *= 60 #resolution
    xdata[x] /= 3.5 #magnitication effect

In [271]:
import statistics as s
for x in range (0,len(xdata)):
    print("Mean:", s.mean(xdata[x]))
    print("Variance: ", s.variance(xdata[x]))
    print("Population stdev: ", s.pstdev(xdata[x]))
    print("Sample stdev: ", s.stdev(xdata[x]))

Mean: 437.14285714285717
Variance:  67493.87755102041
Population stdev:  257.28567462520516
Sample stdev:  259.79583820958413
Mean: 351.42857142857144
Variance:  44228.57142857143
Population stdev:  207.78717526373381
Sample stdev:  210.30589965231937
Mean: 291.42857142857144
Variance:  30857.14285714286
Population stdev:  173.13437037192134
Sample stdev:  175.66201313073597
Mean: 325.7142857142857
Variance:  38204.08163265306
Population stdev:  192.9364916129615
Sample stdev:  195.45864430270936


In [272]:
# Data Mask

for n in range (len(xdata)):
    xmin = icf . find_closest ( xdata[n] , 0 )
    xmax = icf . find_closest ( xdata[n] , 700 )
    print(xmin)
    print(xmax)
    xdata[n] = xdata[n] [ xmin : xmax]
    ydata[n] = ydata[n] [ xmin : xmax ]


0
41
0
41
0
34
0
38


In [273]:
import numpy as np
from scipy.optimize import curve_fit
import icf
p = 6


# This function will generate a perfect Gaussian 
# You can replace this function with any other you want to fit with...
def gaussian(x, *params):

	A = params[0]
	x0 = params[1]
	c = params[2]
	y0 = params[3]
	
	return A*np.exp(-(x-x0)**2/(2*c*c))

def super_gaussian(x, *params):

	A = params[0]
	x0 = params[1]
	c = params[2]
	y0 = params[3]
	
	return y0 + A*np.exp(-((x-x0)/(np.sqrt(2)*c))**(p))


	
yfit  = []
for n in range(0,len(xdata)):
	yfit.append([])
	
#
# This makes a gaussian using these x points
for n in range(len(xdata)):



	#
	# This section will do a fit
	#

	# This does the fit, and returns the fit parameters and the covariances

	guess = [int(np.max(ydata[n])-np.min(ydata[n])),int(xdata[n][np.argmax(ydata[n])]),int(400/(2*np.sqrt(2*np.log(2)))),int(np.min(ydata[n]))]
	print("Our initial guess is", guess)
	popt, pcov = curve_fit(super_gaussian, xdata[n], ydata[n], p0=guess)


	for i in range(len(popt)):
		print ("Parameter",i,":",popt[i],"+/-",np.sqrt(pcov[i][i]))
		
	print("Fit parameters : ", popt)
	print("Fit standard deviations : ", np.sqrt(np.diag(pcov)))


	# This generates a new list with a Gaussian using the identified fit parameters
	# This data is therefore the best fit curve 
	yfit[n] = super_gaussian(xdata[n], *popt)

	print("R^2 = ", icf.r_squared(ydata[n], yfit[n]))

	# This will plot the output, both the original data and the best fit, as well as a residual
	# Note this is a special plotting routine written for the icf labs, hence the 'icf' prefix
	# The source code can be found in icf.py if you want to copy/alter it
	
	icf.fit_plot(xdata[n], ydata[n], yfit[n],xl = "1",yl = "3",title = "felix")

Our initial guess is [4282, 411, 169, 27878]
Parameter 0 : 3214.8627820448437 +/- 103.67620431705758
Parameter 1 : 385.4443023822165 +/- 2.9425178205856195
Parameter 2 : 153.17979439980573 +/- 2.80801342834858
Parameter 3 : 28528.45512138014 +/- 86.09253239730862
Fit parameters :  [ 3214.86278204   385.44430238   153.1797944  28528.45512138]
Fit standard deviations :  [103.67620432   2.94251782   2.80801343  86.0925324 ]
R^2 =  0.9647575303812761


TypeError: fit_plot() got an unexpected keyword argument 'title'

In [ ]:
for n in range (0,len(xdata)):
    plt.plot(xdata[n], ydata[n])
plt.show()
